# Preprocessamento

Este notebook é para tirar os dados do `.csv` e migrar eles para o Postgres.

## 1. Preparando o ambiente

Antes de iniciarmos a execução do projeto, é fundamental preparar o ambiente Python. Este repositório utiliza Pipenv como gerenciador de dependências e ambientes virtuais. Portanto, recomenda-se que você tenha o Pipenv instalado em sua máquina local.

Após confirmar a instalação, execute o seguinte comando na raiz do projeto:

```bash
pipenv install
```

Esse comando irá instalar automaticamente todas as dependências necessárias para a reprodução e execução do projeto.

In [1]:
import csv
import sys
import pandas
import sqlalchemy
import sqlalchemy.orm as orm
import os
import json
import re
import html
import unicodedata
import ast
from dotenv import load_dotenv
from datetime import datetime

In [2]:
# tamanho do batch para inserção no banco
BATCH_SIZE = 2_500

# caminho do arquivo corrigido pelo notebook `01-eda.ipynb`
CSV_FIXED_PATH = "./datasets/games_fixed.csv"

In [3]:
# remove o limite de tamanho de campo do csv
csv.field_size_limit(sys.maxsize)

# mostra todas as colunas do dataframe
pandas.set_option('display.max_columns', None)

## 2. Preparando o PostgreSQL

Antes de migrarmos para o PostgreSQL, é necessário preparar o banco de dados e suas tabelas para receber os dados do projeto.

Recomendamos utilizar o Docker, pois isso simplifica a criação e execução de um contêiner com PostgreSQL na máquina local. Caso ainda não tenha o Docker instalado, consulte as instruções oficiais disponíveis em: https://docs.docker.com/engine/install/

Este repositório inclui um arquivo compose.yml e um arquivo de exemplo .env.example. Para configurar o ambiente:

1. Crie um arquivo .env na raiz do projeto.

2. Copie o conteúdo de .env.example para dentro do novo arquivo .env.

Com isso feito, execute o comando abaixo para inicializar um contêiner com PostgreSQL:

```bash
docker compose up -d
```

Esse comando iniciará o banco de dados em segundo plano, permitindo que você continue utilizando o terminal enquanto o serviço é executado.

### 2.1 Conectando com o Banco de Dados PostgreSQL

Após configurar o ambiente e iniciar o servidor PostgreSQL via Docker, o próximo passo é estabelecer a conexão entre o projeto Python e o banco de dados. Para isso, utilizamos o pacote SQLAlchemy, que fornece uma interface de alto nível para comunicação com bancos relacionais, e o pacote python-dotenv, responsável por carregar as variáveis de ambiente definidas no arquivo `.env`.

#### 2.1.1. Carregando variáveis de ambiente

As credenciais de acesso ao banco (host, porta, usuário, senha e nome do banco) são armazenadas no arquivo `.env`. Isso evita que informações sensíveis fiquem expostas diretamente no código. No início do script, utilizamos:

In [4]:
load_dotenv()

True

Esse comando carrega todas as variáveis definidas no `.env` para o ambiente de execução, permitindo acessá-las via `os.getenv()`.

#### 2.1.2. Validando as configurações

Para garantir que todas as variáveis necessárias foram informadas, o código realiza uma verificação simples. Caso alguma esteja ausente, uma exceção é lançada. Essa etapa evita falhas silenciosas e facilita o diagnóstico de erros de configuração.

In [5]:
# Carregar variáveis de ambiente
POSTGRES_HOST = os.getenv("POSTGRES_HOST")
POSTGRES_PORT = os.getenv("POSTGRES_PORT")
POSTGRES_USER = os.getenv("POSTGRES_USER")
POSTGRES_PASSWORD = os.getenv("POSTGRES_PASSWORD")
POSTGRES_DB = os.getenv("POSTGRES_DB")

# Verificar se todas as variáveis foram carregadas
required_vars = {
    "POSTGRES_HOST": POSTGRES_HOST,
    "POSTGRES_PORT": POSTGRES_PORT,
    "POSTGRES_USER": POSTGRES_USER,
    "POSTGRES_PASSWORD": POSTGRES_PASSWORD,
    "POSTGRES_DB": POSTGRES_DB,
}

missing = [k for k, v in required_vars.items() if not v]
if missing:
    raise EnvironmentError(f"Variáveis de ambiente ausentes: {', '.join(missing)}")

#### 2.1.3. Construindo a URL de conexão

Com as variáveis carregadas, construímos a URL de conexão no formato compatível com o driver `psycopg` do SQLAlchemy:

In [6]:
DATABASE_URL = (
    f"postgresql+psycopg://{POSTGRES_USER}:{POSTGRES_PASSWORD}"
    f"@{POSTGRES_HOST}:{POSTGRES_PORT}/{POSTGRES_DB}"
)

Essa URL contém todas as informações necessárias para que o SQLAlchemy saiba como se conectar ao banco.

#### 2.1.4. Criando a engine do SQLAlchemy

A engine é o objeto central responsável por gerenciar conexões e enviar comandos ao banco:

In [7]:
engine = sqlalchemy.create_engine(DATABASE_URL)

Ela serve como ponto de entrada para operações como consultas, inserções e criação de tabelas.

#### 2.1.5. Testando a conexão

Por fim, o código tenta abrir uma conexão simples com o banco. Caso seja bem-sucedido, uma mensagem de sucesso é exibida; caso contrário, a exceção é capturada e exibida:

In [8]:
try:
    with engine.connect() as connection:
        print("✅ Conexão com o banco de dados estabelecida com sucesso!")
except Exception as e:
    print(f"❌ Erro ao conectar ao banco de dados: {e}")

✅ Conexão com o banco de dados estabelecida com sucesso!


Essa verificação é importante para garantir que o restante do notebook possa interagir com o banco normalmente.

### 2.2 Modelagem do Banco de Dados com SQLAlchemy

A seguir, apresentamos a definição das tabelas e classes que compõem o modelo relacional utilizado neste projeto. A modelagem foi implementada com SQLAlchemy ORM, que permite mapear tabelas e relacionamentos do banco de dados para classes Python, facilitando operações de consulta, inserção e manipulação de dados.

#### 2.2.1. Base declarativa

In [9]:
Base = orm.declarative_base()

O SQLAlchemy utiliza uma base declarativa como ponto central para registrar todas as classes (models) e metadados das tabelas. Todas as entidades do banco herdam dessa `Base`, o que permite ao framework gerar automaticamente as tabelas no PostgreSQL.

#### 2.2.2. Tabelas de associação (relacionamentos M:N)

Muitos dos relacionamentos entre jogos (games) e outros elementos (como desenvolvedores, categorias, gêneros, tags e idiomas) são do tipo muitos-para-muitos (M:N). Em um modelo relacional tradicional, esse tipo de relação deve ser representado por tabelas pivô (também chamadas de tabelas de associação ou junction tables).

Por exemplo, um jogo pode ter vários desenvolvedores, e um desenvolvedor pode estar associado a vários jogos. Esse padrão se repete para publishers, categorias, gêneros, tags e idiomas.

Cada tabela de associação é criada da seguinte forma:

In [10]:
game_developer = sqlalchemy.Table(
    "game_developer",
    Base.metadata,
    sqlalchemy.Column("game_id", sqlalchemy.Integer, sqlalchemy.ForeignKey("games.id"), primary_key=True),
    sqlalchemy.Column("developer_id", sqlalchemy.Integer, sqlalchemy.ForeignKey("developers.id"), primary_key=True)
)

game_publisher = sqlalchemy.Table(
    "game_publisher",
    Base.metadata,
    sqlalchemy.Column("game_id", sqlalchemy.Integer, sqlalchemy.ForeignKey("games.id"), primary_key=True),
    sqlalchemy.Column("publisher_id", sqlalchemy.Integer, sqlalchemy.ForeignKey("publishers.id"), primary_key=True)
)

game_category = sqlalchemy.Table(
    "game_category",
    Base.metadata,
    sqlalchemy.Column("game_id", sqlalchemy.Integer, sqlalchemy.ForeignKey("games.id"), primary_key=True),
    sqlalchemy.Column("category_id", sqlalchemy.Integer, sqlalchemy.ForeignKey("categories.id"), primary_key=True)
)

game_genre = sqlalchemy.Table(
    "game_genre",
    Base.metadata,
    sqlalchemy.Column("game_id", sqlalchemy.Integer, sqlalchemy.ForeignKey("games.id"), primary_key=True),
    sqlalchemy.Column("genre_id", sqlalchemy.Integer, sqlalchemy.ForeignKey("genres.id"), primary_key=True)
)

game_tag = sqlalchemy.Table(
    "game_tag",
    Base.metadata,
    sqlalchemy.Column("game_id", sqlalchemy.Integer, sqlalchemy.ForeignKey("games.id"), primary_key=True),
    sqlalchemy.Column("tag_id", sqlalchemy.Integer, sqlalchemy.ForeignKey("tags.id"), primary_key=True)
)

game_language = sqlalchemy.Table(
    "game_language",
    Base.metadata,
    sqlalchemy.Column("game_id", sqlalchemy.Integer, sqlalchemy.ForeignKey("games.id"), primary_key=True),
    sqlalchemy.Column("language_id", sqlalchemy.Integer, sqlalchemy.ForeignKey("languages.id"), primary_key=True)
)

#### 2.2.3. Características importantes das tabelas pivô:

- Não possuem classe ORM própria — pois servem apenas como ligação entre duas entidades.

- Cada linha representa um vínculo M:N entre duas tabelas.

- Usam chaves estrangeiras para garantir integridade referencial.

- A combinação de colunas funciona como chave primária composta.

São definidas tabelas equivalentes para publishers, categorias, gêneros, tags e idiomas.

#### 2.2.4. Definição dos modelos (tabelas principais)

Cada entidade principal do banco de dados (Developer, Publisher, Category, Genre, Tag, Language, Game, entre outras) é representada por uma classe que herda da `Base`.

In [11]:
class Developer(Base):
    __tablename__ = "developers"

    id = sqlalchemy.Column(sqlalchemy.Integer, primary_key=True)
    name = sqlalchemy.Column(sqlalchemy.Text, nullable=False)

    games = orm.relationship("Game", secondary=game_developer, back_populates="developers")

    @classmethod
    def get_or_create(cls, name, session):
        parsed = name.strip().lower()
        obj = session.query(cls).filter_by(name=parsed).first()
        if obj:
            return obj
        obj = cls(name=parsed)
        session.add(obj)
        session.commit()
        return obj


class Publisher(Base):
    __tablename__ = "publishers"

    id = sqlalchemy.Column(sqlalchemy.Integer, primary_key=True)
    name = sqlalchemy.Column(sqlalchemy.Text, nullable=False)

    games = orm.relationship("Game", secondary=game_publisher, back_populates="publishers")

    @classmethod
    def get_or_create(cls, name, session):
        parsed = name.strip().lower()
        obj = session.query(cls).filter_by(name=parsed).first()
        if obj:
            return obj
        obj = cls(name=parsed)
        session.add(obj)
        session.commit()
        return obj


class Category(Base):
    __tablename__ = "categories"

    id = sqlalchemy.Column(sqlalchemy.Integer, primary_key=True)
    name = sqlalchemy.Column(sqlalchemy.String(255), nullable=False)

    games = orm.relationship("Game", secondary=game_category, back_populates="categories")

    @classmethod
    def get_or_create(cls, name, session):
        parsed = name.strip().lower()
        obj = session.query(cls).filter_by(name=parsed).first()
        if obj:
            return obj
        obj = cls(name=parsed)
        session.add(obj)
        session.commit()
        return obj


class Genre(Base):
    __tablename__ = "genres"

    id = sqlalchemy.Column(sqlalchemy.Integer, primary_key=True)
    name = sqlalchemy.Column(sqlalchemy.String(255), nullable=False)

    games = orm.relationship("Game", secondary=game_genre, back_populates="genres")

    @classmethod
    def get_or_create(cls, name, session):
        parsed = name.strip().lower()
        obj = session.query(cls).filter_by(name=parsed).first()
        if obj:
            return obj
        obj = cls(name=parsed)
        session.add(obj)
        session.commit()
        return obj


class Tag(Base):
    __tablename__ = "tags"

    id = sqlalchemy.Column(sqlalchemy.Integer, primary_key=True)
    name = sqlalchemy.Column(sqlalchemy.String(255), nullable=False)

    games = orm.relationship("Game", secondary=game_tag, back_populates="tags")

    @classmethod
    def get_or_create(cls, name, session):
        parsed = name.strip().lower()
        obj = session.query(cls).filter_by(name=parsed).first()
        if obj:
            return obj
        obj = cls(name=parsed)
        session.add(obj)
        session.commit()
        return obj


class Language(Base):
    __tablename__ = "languages"

    id = sqlalchemy.Column(sqlalchemy.Integer, primary_key=True)
    name = sqlalchemy.Column(sqlalchemy.String(255), nullable=False)

    games = orm.relationship("Game", secondary=game_language, back_populates="languages")

    @classmethod
    def get_or_create(cls, name, session):
        parsed = name.strip().lower()
        obj = session.query(cls).filter_by(name=parsed).first()
        if obj:
            return obj
        obj = cls(name=parsed)
        session.add(obj)
        session.commit()
        return obj


class Screenshot(Base):
    __tablename__ = "screenshots"

    id = sqlalchemy.Column(sqlalchemy.Integer, primary_key=True)
    game_id = sqlalchemy.Column(sqlalchemy.Integer, sqlalchemy.ForeignKey("games.id"), nullable=False)
    screenshot_url = sqlalchemy.Column(sqlalchemy.String(255), nullable=False)


class Movie(Base):
    __tablename__ = "movies"

    id = sqlalchemy.Column(sqlalchemy.Integer, primary_key=True)
    game_id = sqlalchemy.Column(sqlalchemy.Integer, sqlalchemy.ForeignKey("games.id"), nullable=False)
    movie_url = sqlalchemy.Column(sqlalchemy.String(255), nullable=False)


class Game(Base):
    __tablename__ = "games"

    id = sqlalchemy.Column(sqlalchemy.Integer, primary_key=True)
    app_id = sqlalchemy.Column(sqlalchemy.Integer, nullable=False)
    name = sqlalchemy.Column(sqlalchemy.Text, nullable=False)
    release_date = sqlalchemy.Column(sqlalchemy.Date, nullable=False)
    estimated_owners_lower = sqlalchemy.Column(sqlalchemy.Integer, nullable=False)
    estimated_owners_upper = sqlalchemy.Column(sqlalchemy.Integer, nullable=False)
    peak_ccu = sqlalchemy.Column(sqlalchemy.Integer, nullable=False, default=0)
    required_age = sqlalchemy.Column(sqlalchemy.Integer, nullable=False, default=0)
    price = sqlalchemy.Column(sqlalchemy.Float, nullable=False, default=0.0)
    discount = sqlalchemy.Column(sqlalchemy.Float, nullable=False, default=0.0)
    dlc_count = sqlalchemy.Column(sqlalchemy.Integer, nullable=False, default=0)
    about_the_game = sqlalchemy.Column(sqlalchemy.Text, nullable=True)
    header_image = sqlalchemy.Column(sqlalchemy.Text, nullable=True)
    website = sqlalchemy.Column(sqlalchemy.Text, nullable=True)
    support_url = sqlalchemy.Column(sqlalchemy.Text, nullable=True)
    support_email = sqlalchemy.Column(sqlalchemy.Text, nullable=True)
    windows = sqlalchemy.Column(sqlalchemy.Boolean, nullable=False, default=False)
    mac = sqlalchemy.Column(sqlalchemy.Boolean, nullable=False, default=False)
    linux = sqlalchemy.Column(sqlalchemy.Boolean, nullable=False, default=False)
    metacritic_score = sqlalchemy.Column(sqlalchemy.Float, nullable=True)
    metacritic_url = sqlalchemy.Column(sqlalchemy.Text, nullable=True)
    user_score = sqlalchemy.Column(sqlalchemy.Float, nullable=True)
    positive = sqlalchemy.Column(sqlalchemy.Integer, nullable=True)
    negative = sqlalchemy.Column(sqlalchemy.Integer, nullable=True)
    score_rank = sqlalchemy.Column(sqlalchemy.Integer, nullable=True)
    achievements = sqlalchemy.Column(sqlalchemy.Integer, nullable=True)
    recommendations = sqlalchemy.Column(sqlalchemy.Integer, nullable=True)
    average_playtime_forever = sqlalchemy.Column(sqlalchemy.Integer, nullable=True)
    average_playtime_2weeks = sqlalchemy.Column(sqlalchemy.Integer, nullable=True)
    median_playtime_forever = sqlalchemy.Column(sqlalchemy.Integer, nullable=True)
    median_playtime_2weeks = sqlalchemy.Column(sqlalchemy.Integer, nullable=True)

    # RELACIONAMENTOS M:N
    developers = orm.relationship("Developer", secondary=game_developer, back_populates="games")
    publishers = orm.relationship("Publisher", secondary=game_publisher, back_populates="games")
    categories = orm.relationship("Category", secondary=game_category, back_populates="games")
    genres = orm.relationship("Genre", secondary=game_genre, back_populates="games")
    tags = orm.relationship("Tag", secondary=game_tag, back_populates="games")
    languages = orm.relationship("Language", secondary=game_language, back_populates="games")


#### 2.2.5. Método `get_or_create`

As classes também implementam um método utilitário:

```python
@classmethod
def get_or_create(cls, name, session):
```

Esse método:

- Padroniza o nome (ex.: minúsculas).

- Verifica se o registro já existe.

- Caso não exista, cria um novo.

- Garante consistência dos dados ao evitar duplicações.

### 2.3. Criando as Tabelas no Banco de Dados e Inicializando a Sessão

Após definir todos os modelos e tabelas com o SQLAlchemy, o próximo passo é criar a estrutura física dessas tabelas dentro do banco PostgreSQL e preparar uma sessão para realizar operações de leitura e escrita.

#### 2.3.1 Criando as tabelas no banco

In [12]:
Base.metadata.create_all(engine)

O SQLAlchemy mantém, por meio de Base.metadata, um registro completo de todas as classes (models) e tabelas declaradas no projeto. Ao chamar create_all(engine), o framework:

- Verifica quais tabelas já existem no banco.

- Cria automaticamente apenas as tabelas que ainda não foram criadas.

- Garante a criação das tabelas pivô, modelos principais e relacionamentos.

Esse comando funciona como uma migração inicial simplificada, útil especialmente em desenvolvimento ou quando o esquema do banco ainda não está sendo gerenciado por uma ferramenta de migração mais avançada.

#### 2.3.2 Criando a sessão de comunicação com o banco

In [13]:
session = sqlalchemy.orm.Session(engine)

A sessão (Session) é o componente central do SQLAlchemy ORM para interação com o banco. Ela funciona como:

- Uma unidade de trabalho (unit of work), controlando transações.

- Um gerenciador de objetos, mantendo instâncias do banco sincronizadas com o Python.

- A interface por onde realizamos:

  - consultas (session.query(...))

  - inserções (session.add(...))

  - atualizações

  - deleções

  - commits e rollbacks

## 3. Migrar dados

A etapa de migração consiste em transformar os dados extraídos do dataset original para o formato necessário no banco PostgreSQL. Neste notebook, a migração é realizada gradualmente para cada entidade relacionada aos jogos.

### 3.1. Carregando o Dataset com Pandas

O dataset bruto é carregado utilizando o Pandas:

In [14]:
games_dataset = pandas.read_csv(
  CSV_FIXED_PATH,
  sep=",",
  quotechar='"',
  quoting=csv.QUOTE_MINIMAL,
  engine="python",
  encoding="utf-8-sig",
)

Esse procedimento lê o arquivo CSV original contendo informações sobre milhares de jogos e suas respectivas categorias, incluindo as tags. Cada coluna será posteriormente transformada para se adequar ao esquema definido no SQLAlchemy.

### 3.2. Processando e Migrando as Tags

A coluna Tags do dataset contém listas de tags separadas por vírgulas. Antes de armazená-las, é necessário realizar três etapas:

1. Identificar todas as tags únicas existentes no dataset.

2. Criar no banco apenas aquelas que ainda não foram registradas.

3. Substituir o texto original da coluna por listas de IDs, permitindo relacioná-las aos jogos futuramente.

#### 3.2.1 Extração das tags únicas

In [15]:
# pega todas as tags da coluna, remove NaN e garante strings
tag_column = games_dataset['tags'].dropna().astype(str)

unique_tags = set()

# extrai tags únicas
for tag_string in tag_column:
    names = (name.strip().lower() for name in tag_string.split(','))
    unique_tags.update(names)

Aqui:

- Removemos valores ausentes (NaN).

- Garantimos que todas as entradas sejam strings.

- Dividimos cada linha em múltiplas tags.

- Normalizamos (removendo espaços e padronizando para lowercase).

- Armazenamos todas as tags únicas em um conjunto (set), que evita duplicações.

#### 3.2.2 Verificando tags já existentes no banco

In [16]:
existing_tags = {
    t.name.lower()
    for t in session.query(Tag).all()
}

Essa consulta evita inserir tags duplicadas, garantindo consistência entre o CSV e o banco.

#### 3.2.3 Inserindo apenas tags novas

In [17]:
new_tags = [
    Tag(name=tag)
    for tag in unique_tags
    if tag not in existing_tags
]

session.bulk_save_objects(new_tags)
session.commit()

**Por que `bulk_save_objects`?**

Porque esse método é significativamente mais rápido que inserir objeto por objeto, principalmente quando lidamos com centenas ou milhares de registros.

Após a inserção, consultamos todas as tags novamente e criamos um mapa de acesso rápido:

In [18]:
tag_map = {
    tag.name.lower(): tag.id
    for tag in session.query(Tag).all()
}

Essa estrutura permite converter o texto das tags em IDs inteiros, que são as chaves estrangeiras necessárias para preencher a tabela de associação `game_tag`.

#### 3.2.4 Função auxiliar para mapear nomes → IDs

In [19]:
def get_tag_id(tag_names):
    if not isinstance(tag_names, str):
        return None
    
    ids = []
    for name in map(str.strip, tag_names.lower().split(',')):
        tag_id = tag_map.get(name)
        if tag_id is None:
            print(f"Tag {name} não encontrada")
        else:
            ids.append(tag_id)
    return ids

Essa função:

- Recebe um texto com múltiplas tags.

- Normaliza cada tag.

- Procura seu ID no dicionário tag_map.

- Retorna uma lista contendo todos os IDs encontrados.

Essa lista servirá diretamente para construir os relacionamentos M:N posteriormente.

#### 3.2.5 Substituindo a coluna original por IDs

In [20]:
games_dataset['tags'] = games_dataset['tags'].apply(get_tag_id)

Agora, a coluna Tags não contém mais strings, mas listas de IDs, no padrão que será utilizado para inserir corretamente os dados dos jogos e suas relações no banco.

### 3.2. Desenvolvedores (developers)

O processo de migração dos desenvolvedores segue a mesma lógica apresentada na seção anterior (Migração das Tags). A coluna Developers contém múltiplos valores por jogo, separados por vírgulas, exigindo tratamento semelhante para identificar valores únicos e associá-los aos registros correspondentes.

In [21]:
# pega todos os developers da coluna
developer_column = games_dataset['developers'].dropna().astype(str)

# usa set para evitar duplicatas
unique_developers = set()

# extrai todos os developers únicos
for dev_string in developer_column:
    names = (name.strip().lower() for name in dev_string.split(','))
    unique_developers.update(names)

# pega do banco de dados os developers já existentes
existing = {
    d.name.lower()
    for d in session.query(Developer).all()
}

# filtra apenas os novos
new_developers = [
    Developer(name=name)
    for name in unique_developers
    if name not in existing
]

# Inserção em lote (melhor desempenho do que inserir individualmente)
for i in range(0, len(new_developers), BATCH_SIZE):
    batch = new_developers[i:i+BATCH_SIZE]
    session.add_all(batch)
    session.commit()

# Cria mapa {nome: id} para facilitar associação futura
developer_map = {
    dev.name.lower(): dev.id
    for dev in session.query(Developer).all()
}

# Função que converte lista de nomes em lista de IDs
def get_developer_id(developer_names):
    if not isinstance(developer_names, str):
        return None
    
    ids = []
    for name in map(str.strip, developer_names.lower().split(',')):
        dev_id = developer_map.get(name)
        if dev_id is None:
            print(f"Developer {name} não encontrado")
        else:
            ids.append(dev_id)
    return ids

# Substitui a coluna original pelas listas de IDs
games_dataset['developers'] = games_dataset['developers'].apply(get_developer_id)

### 3.3. Publicadores (publishers)

A migração dos publishers segue a mesma estrutura descrita anteriormente para Tags e Developers. Assim, aplicamos novamente um fluxo semelhante.

In [22]:
# Pega todos os valores da coluna
publisher_column = games_dataset['publishers'].dropna().astype(str)

unique_publishers = set()

# extrai todos os publishers únicos
for pub_string in publisher_column:
    names = (name.strip().lower() for name in pub_string.split(','))
    unique_publishers.update(names)

# pega do banco de dados os publishers já existentes
existing = {
    p.name.lower()
    for p in session.query(Publisher).all()
}

# filtra apenas os novos
new_publishers = [
    Publisher(name=name)
    for name in unique_publishers
    if name not in existing
]

# Inserção em lote (melhor desempenho do que inserir individualmente)
for i in range(0, len(new_publishers), BATCH_SIZE):
    batch = new_publishers[i:i+BATCH_SIZE]
    session.add_all(batch)
    session.commit()

# Cria mapa {nome: id} para facilitar associação futura
publisher_map = {
    pub.name.lower(): pub.id
    for pub in session.query(Publisher).all()
}

# Função que converte lista de nomes em lista de IDs
def get_publisher_id(publisher_names):
    if not isinstance(publisher_names, str):
        return None
    
    ids = []
    for name in map(str.strip, publisher_names.lower().split(',')):
        pub_id = publisher_map.get(name)
        if pub_id is None:
            print(f"Publisher {name} não encontrado")
        else:
            ids.append(pub_id)
    return ids

# Substitui a coluna original pelas listas de IDs
games_dataset['publishers'] = games_dataset['publishers'].apply(get_publisher_id)

### 3.4. Categorias (categories)

A migração das categorias segue o mesmo padrão já utilizado para Tags, Developers e Publishers. Assim, aplicamos novamente as etapas habituais:

In [23]:
# pega todos os valores da coluna
categories_column = games_dataset['categories'].dropna().astype(str)

unique_categories = set()

# extrai todos os categories únicos
for cat_string in categories_column:
    names = (name.strip().lower() for name in cat_string.split(','))
    unique_categories.update(names)

# pega do banco de dados os categories já existentes
existing = {
    c.name.lower()
    for c in session.query(Category).all()
}

# filtra apenas os novos
new_categories = [
    Category(name=name)
    for name in unique_categories
    if name not in existing
]

# insere em lote (muito mais rápido do que inserir um a um)
for i in range(0, len(new_categories), BATCH_SIZE):
    batch = new_categories[i:i+BATCH_SIZE]
    session.add_all(batch)
    session.commit()

category_map = {
    cat.name.lower(): cat.id
    for cat in session.query(Category).all()
}

def get_category_id(category_names):
    if not isinstance(category_names, str):
        return None
    
    ids = []
    for name in map(str.strip, category_names.lower().split(',')):
        cat_id = category_map.get(name)
        if cat_id is None:
            print(f"Category {name} não encontrada")
        else:
            ids.append(cat_id)
    return ids

# aplica ao dataset
games_dataset['categories'] = games_dataset['categories'].apply(get_category_id)

### 3.5. Generos (genres)

> *Como o processo de migração de atributos multivalorados já foi descrito detalhadamente nas seções anteriores, esta etapa apresenta apenas um resumo das operações realizadas.*

In [24]:
# Pega todos os valores da coluna
genres_column = games_dataset['genres'].dropna().astype(str)

unique_genres = set()

# extrai todos os genres únicos
for genre_string in genres_column:
    names = (name.strip().lower() for name in genre_string.split(','))
    unique_genres.update(names)

# Pega do banco de dados os genres já existentes
existing = {
    g.name.lower()
    for g in session.query(Genre).all()
}

# filtra apenas os novos
new_genres = [
    Genre(name=name)
    for name in unique_genres
    if name not in existing
]

# insere em lote (muito mais rápido do que inserir um a um)
for i in range(0, len(new_genres), BATCH_SIZE):
    batch = new_genres[i:i+BATCH_SIZE]
    session.add_all(batch)
    session.commit()

genre_map = {
    genre.name.lower(): genre.id
    for genre in session.query(Genre).all()
}

def get_genre_id(genre_names):
    if not isinstance(genre_names, str):
        return None
    
    ids = []
    for name in map(str.strip, genre_names.lower().split(',')):
        genre_id = genre_map.get(name)
        if genre_id is None:
            print(f"Genre {name} não encontrado")
        else:
            ids.append(genre_id)
    return ids

# aplica ao dataset
games_dataset['genres'] = games_dataset['genres'].apply(get_genre_id)

### 3.6. Linguagens

Diferentemente dos atributos anteriores, a migração das linguagens requer um processamento muito mais elaborado. No dataset da Steam, a coluna Supported languages apresenta dados altamente inconsistentes, contendo:

- listas malformadas (com aspas quebradas, colchetes incompletos, mistura de ' e "),

- HTML embutido (`<br>`, `<strong>`, etc.),

- marcações BBCode (`[b]`, `[i]`),

- caracteres estranhos (&lt, &gt, &amp),

- idiomas concatenados sem separação clara,

- variantes inconsistentes de nomes de idiomas ("Chinese Simplified Text Only" → "simplified chinese").

Por isso, o processo foi dividido em diversas etapas de limpeza e normalização antes da migração propriamente dita.

#### 3.6.1 Correção estrutural de listas malformadas

Algumas entradas vêm com formato próximo de JSON, mas inválido:

```json
["English, French, German]
['Portuguese', 'Spanish']
[English, Japanese]
```

A função `safe_parse_languages` executa várias estratégias:

1. Tentar interpretar a string como JSON válida.

2. Tentar o ast.literal_eval, que aceita sintaxes Python-like.

3. Corrigir manualmente itens quebrados:

  1. substituir aspas simples → duplas,

  2. colocar aspas em itens ausentes,

  3. ajustar colchetes mal formados.

6. Repetir a tentativa de JSON.

7. Caso ainda falhe, utilizar fallback simples: dividir por vírgulas.

O objetivo é garantir que cada entrada seja transformada em uma lista de strings representando idiomas.

In [25]:
def fix_brackets(m):
    items = m.group(1).split(",")
    items = [f'"{i.strip().strip("\"")}"' for i in items]
    return "[" + ",".join(items) + "]"

def safe_parse_languages(s):
    original = s.strip()

    # 1. Tentar JSON direto (apenas se começar com [)
    if original.startswith("["):
        try:
            return json.loads(original)
        except:
            pass

    # 2. Tentar literal_eval direto
    try:
        return ast.literal_eval(original)
    except:
        pass

    # 3. Tentar corrigir strings malformadas
    fixed = original

    # 3.1 Trocar aspas simples por duplas
    fixed = fixed.replace("'", '"')

    # 3.2 Garantir que itens sem aspas fiquem entre aspas
    # ex: K"iche" -> "K\"iche\""
    fixed = re.sub(r'(\w+)"', r'"\1"', fixed)

    # 3.3 Garantir que itens isolados fiquem entre aspas
    # ex: [English, French] → ["English", "French"]
    fixed = re.sub(r"\[(.*?)\]", fix_brackets, fixed)

    # 4. Tentar JSON novamente após correções
    try:
        return json.loads(fixed)
    except:
        pass

    # 5. Fallback manual — remove colchetes e divide por vírgula
    fallback = original.strip("[]").split(",")
    fallback = [x.strip().strip('"').strip("'") for x in fallback]
    return fallback


#### 3.6.2. Limpeza profunda de cada nome de idioma

A função clean_language aplica um pipeline completo de normalização:

1. Remoção de sujeiras estruturais

  - decodificação de HTML: `&amp;`, `&gt;`, etc.

  - remoção de tags HTML (`<br>`) e BBCode (`[b]`, `[i]`).

  - remoção de símbolos no início e fim.

  - substituição de quebras de linha por vírgulas.

2. Separação de itens grudados

Algumas entradas juntam idiomas:

```arduino
"english dutch english"
```

ou

```arduino
"english|german"  
"japanese/french"
```

A função divide esses casos usando `,` `|` `;` `/` como delimitadores.

3. Normalização textual

- conversão para lowercase,

- limpeza de duplicações internas (“english english” → “english”),

- normalização Unicode (NFKC) para corrigir acentos,

- correções específicas ("simplified chinese text only").

4. Tratamento de casos especiais

- linguagens com apóstrofo preservado (“k'iche'”),

- idiomas com parênteses mal formados,

- remoção de hashtags e lixo residual.

Ao final, cada entrada retorna uma lista de idiomas devidamente normalizados.

In [26]:
def clean_language(raw):
    if not isinstance(raw, str):
        return []

    # ---- 1) Decode de HTML entities ----
    text = html.unescape(raw)

    # ---- 2) Remover tags HTML e BBCode ----
    text = re.sub(r'<[^>]*>', '', text)
    text = re.sub(r'\[/?[a-zA-Z0-9]+\]', '', text)

    # ---- 3) Trocar quebras de linha por vírgula ----
    text = text.replace("\r", ",").replace("\n", ",")

    # ---- 4) Separar itens que vêm grudados ----
    parts = re.split(r'[,\|;/]+', text)

    cleaned = []

    for item in parts:
        item = item.strip().lower()

        if not item:
            continue

        # Remover hashtags (#lang_français)
        if item.startswith("#"):
            continue

        # Remover sobras de HTML mal formadas (lt, gt, amp)
        item = re.sub(r'\b(lt|gt|amp|strong)\b', '', item)
        item = item.replace("&lt", "").replace("&gt", "").replace("&amp", "")

        # Remover símbolos no começo/fim
        item = re.sub(r'^[^a-z0-9]+|[^a-z0-9]+$', '', item)

        # Normalizar Unicode (corrige francês → français)
        item = unicodedata.normalize("NFKC", item)

        # Recolocar idiomas compostos comuns
        item = item.replace("simplified chinese text only", "simplified chinese")
        item = item.replace("traditional chinese text only", "traditional chinese")

        # Remover duplicações internas
        item = re.sub(r'\b(\w+)\s+\1\b', r'\1', item)

        # Tratar casos como english dutch english
        words = item.split()
        if len(words) > 1 and all(w.isalpha() for w in words):
            # Se for uma sequência de idiomas sem vírgula, quebrar
            for w in words:
                cleaned.append(w)
            continue

        # Arrumar k'iche (sem remover apóstrofo)
        if "k'iche" in item:
            item = "k'iche'"

        # Arrumar idiomas que ficaram sem ')'
        if "(" in item and ")" not in item:
            item += ")"  

        # Descartar se ficou vazio
        item = item.strip()
        if item:
            cleaned.append(item)

    return cleaned

#### 3.6.3. Extração das linguagens únicas

Depois de aplicar safe_parse_languages + clean_language, construímos um conjunto de linguagens únicas:

In [27]:
unique_languages = set()

Este conjunto agora contém apenas idiomas limpos, padronizados e coerentes.

#### 3.6.4. Inserção das linguagens no banco

Assim como nas seções anteriores, consultamos linguagens já existentes no banco e inserimos apenas as novas usando batches (`BATCH_SIZE`):

In [28]:
languages_column = games_dataset['supported_languages'].dropna().astype(str)

for lang_string in languages_column:
    lang_list = safe_parse_languages(lang_string)

    for name in lang_list:
        for cleaned in clean_language(name):
            if cleaned:
                unique_languages.add(cleaned)

sorted_unique_languages = sorted(unique_languages)

# pega do banco de dados os languages já existentes
existing = {
    l.name.lower()
    for l in session.query(Language).all()
}

# filtra apenas os novos
new_languages = [
    Language(name=name)
    for name in unique_languages
    if name not in existing
]

# insere em lote (muito mais rápido do que inserir um a um)
for i in range(0, len(new_languages), BATCH_SIZE):
    batch = new_languages[i:i+BATCH_SIZE]
    session.add_all(batch)
    session.commit()

language_map = {
    language.name.lower(): language.id
    for language in session.query(Language).all()
}

def get_language_id(language_names):
    if not isinstance(language_names, str):
        return None
    
    ids = []
    # nome das linguagens estão muito sujos no dataset, então precisamos limpar antes de aplicar o map
    lang_list = safe_parse_languages(language_names)
    for name in lang_list:
        for cleaned in clean_language(name):
            if cleaned:
                lang_id = language_map.get(cleaned)
                if lang_id is None:
                    print(f"Language {cleaned} not found")
                else:
                    ids.append(lang_id)
    return ids

games_dataset['supported_languages'] = games_dataset['supported_languages'].apply(get_language_id)

### 3.7. Migração dos Jogos (Games) para o Banco de Dados

A migração dos jogos representa a etapa central deste pipeline, pois envolve transformar uma grande quantidade de atributos heterogêneos do dataset da Steam em registros consistentes no modelo relacional. Cada linha do dataset é convertida em uma instância da classe Game, que posteriormente será persistida no banco.

Como o dataset original apresenta inconsistências estruturais, valores faltantes, formatações variadas e campos semipreenchidos, o processo de conversão precisa ser feito com cuidado para garantir integridade dos dados.

#### 3.7.1 Processamento de campos que exigem transformação

**Conversão da faixa de Estimated owners**

O dataset traz estimativas de jogadores em formato textual:

- `"1000000 - 2000000"`

- `"0 - 1000"`

- `"5000000"` (caso incompleto)

- `" - 1000"` ou `"1000000 - "` (valor ausente)

A função `get_estimated_owners()` separa a string em valor mínimo e máximo, normaliza campos vazios para `0`, converte ambos os valores para `int` e retorna a tupla `(lower, upper)`.

Isso permite registrar a faixa estimada como dois campos inteiros no banco.

In [29]:
def get_estimated_owners(estimated_owners):
    # break the string into lower and upper
    lower, upper = estimated_owners.split('-')
    lower = lower.strip()
    upper = upper.strip()

    if lower == '':
        lower = 0
    if upper == '':
        upper = 0

    return int(lower), int(upper)

**Padronização das datas (Release date)**

A Steam fornece datas em mais de um formato, por exemplo:

- `"Aug 12, 2022"`

- `"Aug 2023"`

- `"Coming soon"`

A função `parse_steam_date()` tenta converter datas completas `("%b %d, %Y")`, converter datas apenas com mês e ano `("%b %Y")` e retornar None caso não seja possível interpretar a data.

Assim, mantemos coerência nos tipos, armazenando valores inválidos como NULL no banco.

In [30]:
def parse_steam_date(s):
    s = s.strip()

    if not s or s.lower() == "coming soon":
        return None

    try:
        return datetime.strptime(s, "%b %d, %Y").date()
    except ValueError:
        # Steam às vezes usa formatos diferentes, exemplo: "Aug 2023"
        try:
            return datetime.strptime(s, "%b %Y").date()
        except:
            return None

#### 3.7.2 Construção do objeto Game

Vários campos do dataset podem estar vazios, como:

- descrição do jogo,

- imagem de cabeçalho,

- website,

- URL de suporte,

- e-mail de suporte,

- metacritic score,

- score rank.

Como strings vazias ou `"nan"` não são adequadas para um banco relacional, aplicamos esta regra:

- Se o valor estiver vazio, for a string `"nan"` ou identificado como `NaN` pelo Pandas, substituímos por `None`.

Essa padronização melhora robustez e evita inconsistências nos modelos.

In [31]:
# para cada linha do dataset, cria um novo jogo no postgres
database_games = []
for index, row in games_dataset.iterrows():
    id = row['appid']
    name = row['name']

    # converte a data de lançamento para o formato date
    release_date = parse_steam_date(row['release_date'])

    estimated_owners = row['estimated_owners']
    estimated_owners_lower, estimated_owners_upper = get_estimated_owners(estimated_owners)
    peak_ccu = row['peak_ccu']
    required_age = row['required_age']
    price = row['price']
    discount = row['discount']
    dlc_count = row['dlc_count']

    # se about_the_game estiver vazio, define como None
    about_the_game = row['about_the_game']
    if about_the_game == '' or about_the_game == 'nan' or pandas.isna(about_the_game):
        about_the_game = None

    # se header_image estiver vazio, define como None
    header_image = row['header_image']
    if header_image == '' or header_image == 'nan' or pandas.isna(header_image):
        header_image = None

    # se website estiver vazio, define como None
    website = row['website']
    if website == '' or website == 'nan' or pandas.isna(website):
        website = None

    # se support_url estiver vazio, define como None
    support_url = row['support_url']
    if support_url == '' or support_url == 'nan' or pandas.isna(support_url):
        support_url = None

    # se support_email estiver vazio, define como None
    support_email = row['support_email']
    if support_email == '' or support_email == 'nan' or pandas.isna(support_email):
        support_email = None

    windows = row['windows']
    mac = row['mac']
    linux = row['linux']

    # se metacritic_score estiver vazio, define como None
    metacritic_score = row['metacritic_score']
    if metacritic_score == '' or metacritic_score == 'nan' or pandas.isna(metacritic_score):
        metacritic_score = None

    # se metacritic_url estiver vazio, define como None
    metacritic_url = row['metacritic_url']
    if metacritic_url == '' or metacritic_url == 'nan' or pandas.isna(metacritic_url):
        metacritic_url = None

    user_score = row['user_score']
    positive = row['positive']
    negative = row['negative']

    # se score_rank estiver vazio, define como None
    score_rank = row['score_rank']
    if score_rank == '' or score_rank == 'nan' or pandas.isna(score_rank):
        score_rank = None

    achievements = row['achievements']
    recommendations = row['recommendations']
    average_playtime_forever = row['average_playtime_forever']
    average_playtime_2weeks = row['average_playtime_two_weeks']
    median_playtime_forever = row['median_playtime_forever']
    median_playtime_2weeks = row['median_playtime_two_weeks']

    game = Game(
        app_id=id,
        name=name,
        release_date=release_date,
        estimated_owners_lower=estimated_owners_lower,
        estimated_owners_upper=estimated_owners_upper,
        peak_ccu=peak_ccu,
        required_age=required_age,
        price=price,
        discount=discount,
        dlc_count=dlc_count,
        about_the_game=about_the_game,
        header_image=header_image,
        website=website,
        support_url=support_url,
        support_email=support_email,
        windows=windows,
        mac=mac,
        linux=linux,
        metacritic_score=metacritic_score,
        metacritic_url=metacritic_url,
        user_score=user_score,
        positive=positive,
        negative=negative,
        score_rank=score_rank,
        achievements=achievements,
        recommendations=recommendations,
        average_playtime_forever=average_playtime_forever,
        average_playtime_2weeks=average_playtime_2weeks,
        median_playtime_forever=median_playtime_forever,
        median_playtime_2weeks=median_playtime_2weeks,
    )

    database_games.append(game)


#### 3.7.3 Inserção em lote no banco

A quantidade de jogos no dataset pode ser alta (dezenas de milhares). Por isso, a inserção é feita em batches:

In [32]:
# insere em lotes
for i in range(0, len(database_games), BATCH_SIZE):
    batch = database_games[i:i+BATCH_SIZE]
    session.add_all(batch)
    session.commit()

### 3.8. Vinculação dos jogos com atributos

Após inserir todas as entidades principais (Games, Developers, Publishers, Genres, Categories, Tags, Languages) no banco de dados, o próximo passo consiste em criar as relações muitos-para-muitos (M:N) entre os jogos e seus respectivos atributos.

Essas relações são representadas pelas tabelas de associação (association tables) criadas no modelo ORM, como `game_developer`, `game_publisher`, `game_genre`, etc.

A seguir, apresentamos o processo para vincular jogos a desenvolvedores, que serve como modelo geral para as demais associações.

#### 3.8.1 Vinculando Jogos e Desenvolvedores

Cada jogo pode ter um ou mais desenvolvedores, e cada desenvolvedor pode ter contribuído para vários jogos. Isso caracteriza uma relação M:N, implementada pela tabela intermediária `game_developer`.

O processo de vinculação envolve duas etapas fundamentais:

1. Carregar os dados existentes do banco

In [33]:
# 1. Carregar all developers e games
developers = {d.id: d for d in session.query(Developer).all()}
games = {g.app_id: g for g in session.query(Game).all()}

Também carregamos todos os pares já presentes na tabela de associação. Isso evita a criação de vínculos duplicados:

In [34]:
# (game_id, developer_id)
existing_pairs = set()

# tabela de associação
association_table = Game.__table__.metadata.tables["game_developer"]

# carrega os pares já existentes no banco
rows = session.execute(association_table.select())
for game_id, developer_id in rows:
    existing_pairs.add((game_id, developer_id))

2. Identificar os relacionamentos que precisam ser criados e inserção em lote dos novos pares

In [35]:
batch = []
total_inserted = 0

# percorrer o dataset e criar os pares
for row in games_dataset.itertuples():
    game = games.get(row.appid)
    if not game or not isinstance(row.developers, list):
        continue

    for dev_id in row.developers:
        if dev_id not in developers:
            continue

        pair = (game.id, dev_id)
        if pair in existing_pairs:
            continue  # já existe, pula

        existing_pairs.add(pair)  # marcar como já incluído
        batch.append({"game_id": game.id, "developer_id": dev_id})

        if len(batch) >= BATCH_SIZE:
            session.execute(sqlalchemy.insert(association_table), batch)
            total_inserted += len(batch)
            batch.clear()

# inserir o último batch
if batch:
    session.execute(sqlalchemy.insert(association_table), batch)
    total_inserted += len(batch)
    batch.clear()

# commit
session.commit()

#### 3.8.2 Vinculando Jogos e Publicadores

A vinculação entre jogos e publicadores segue exatamente o mesmo procedimento descrito na seção anterior (Games–Developers). Aqui, novamente estamos lidando com uma relação muitos-para-muitos (M:N), representada pela tabela de associação `game_publisher`.

Como este processo é estruturalmente igual ao da vinculação com desenvolvedores, apresentamos aqui apenas esta explicação concisa, mantendo o código como documentação operacional.

In [36]:
# Link publishers to games
association_table = Game.__table__.metadata.tables["game_publisher"]

# 1. Carregar all publishers e games
publishers = {p.id: p for p in session.query(Publisher).all()}
games = {g.app_id: g for g in session.query(Game).all()}

# 2. Criar tracking para evitar duplicações
existing_pairs = set()   # (game_id, publisher_id)

# Também é útil carregar os pares já existentes no banco:
rows = session.execute(association_table.select())
for game_id, publisher_id in rows:
    existing_pairs.add((game_id, publisher_id))

batch = []
total_inserted = 0

# 3. Processar dataset
for row in games_dataset.itertuples():
    game = games.get(row.appid)
    if not game or not isinstance(row.publishers, list):
        continue

    for pub_id in row.publishers:
        if pub_id not in publishers:
            continue

        pair = (game.id, pub_id)
        if pair in existing_pairs:
            continue  # já existe, pula

        existing_pairs.add(pair)  # marcar como já incluído
        batch.append({"game_id": game.id, "publisher_id": pub_id})

        if len(batch) >= BATCH_SIZE:
            session.execute(sqlalchemy.insert(association_table), batch)
            total_inserted += len(batch)
            batch.clear()

# 4. Inserir último batch
if batch:
    session.execute(sqlalchemy.insert(association_table), batch)
    total_inserted += len(batch)
    batch.clear()

session.commit()

#### 3.8.3 Vinculando Jogos e Categorias

Vincular jogos com categorias segue um procedimento muito semelhante aos anteriores.

In [37]:
association_table = Game.__table__.metadata.tables["game_category"]

# 1. Carregar all categories e games
categories = {c.id: c for c in session.query(Category).all()}
games = {g.app_id: g for g in session.query(Game).all()}

# 2. Criar tracking para evitar duplicações
existing_pairs = set()   # (game_id, category_id)

# Também é útil carregar os pares já existentes no banco:
rows = session.execute(association_table.select())
for game_id, category_id in rows:
    existing_pairs.add((game_id, category_id))

batch = []
total_inserted = 0

# 3. Processar dataset
for row in games_dataset.itertuples():
    game = games.get(row.appid)
    if not game or not isinstance(row.categories, list):
        continue

    for cat_id in row.categories:
        if cat_id not in categories:
            continue
        
        pair = (game.id, cat_id)
        if pair in existing_pairs:
            continue  # já existe, pula

        existing_pairs.add(pair)  # marcar como já incluído
        batch.append({"game_id": game.id, "category_id": cat_id})
        
        if len(batch) >= BATCH_SIZE:
            session.execute(sqlalchemy.insert(association_table), batch)
            total_inserted += len(batch)
            batch.clear()

# 4. Inserir último batch
if batch:
    session.execute(sqlalchemy.insert(association_table), batch)
    total_inserted += len(batch)
    batch.clear()

session.commit()

#### 3.8.4 Vinculando Jogos e Gêneros

Vincular jogos com gêneros segue um procedimento muito semelhante aos anteriores.

In [38]:
association_table = Game.__table__.metadata.tables["game_genre"]

# 1. Carregar all genres e games
genres = {g.id: g for g in session.query(Genre).all()}
games = {g.app_id: g for g in session.query(Game).all()}

existing_pairs = set()   # (game_id, genre_id)

rows = session.execute(association_table.select())
for game_id, genre_id in rows:
    existing_pairs.add((game_id, genre_id))

batch = []
total_inserted = 0

for row in games_dataset.itertuples():
    game = games.get(row.appid)
    if not game or not isinstance(row.genres, list):
        continue
    
    for genre_id in row.genres:
        if genre_id not in genres:
            continue
        
        pair = (game.id, genre_id)
        if pair in existing_pairs:
            continue  # já existe, pula
        
        existing_pairs.add(pair)  # marcar como já incluído
        batch.append({"game_id": game.id, "genre_id": genre_id})
        
        if len(batch) >= BATCH_SIZE:
            session.execute(sqlalchemy.insert(association_table), batch)
            total_inserted += len(batch)
            batch.clear()

if batch:
    session.execute(sqlalchemy.insert(association_table), batch)
    total_inserted += len(batch)
    batch.clear()

session.commit()

#### 3.8.5 Vinculando Jogos e Tags

Vincular jogos com tags segue um procedimento muito semelhante aos anteriores.

In [39]:
association_table = Game.__table__.metadata.tables["game_tag"]

# 1. Carregar all tags e games
tags = {t.id: t for t in session.query(Tag).all()}
games = {g.app_id: g for g in session.query(Game).all()}

existing_pairs = set()   # (game_id, tag_id)

rows = session.execute(association_table.select())
for game_id, tag_id in rows:
    existing_pairs.add((game_id, tag_id))

batch = []
total_inserted = 0

for row in games_dataset.itertuples():
    game = games.get(row.appid)
    if not game or not isinstance(row.tags, list):
        continue
    
    for tag_id in row.tags:
        if tag_id not in tags:
            continue
        
        pair = (game.id, tag_id)
        if pair in existing_pairs:
            continue  # já existe, pula
        
        existing_pairs.add(pair)  # marcar como já incluído
        batch.append({"game_id": game.id, "tag_id": tag_id})
        
        if len(batch) >= BATCH_SIZE:
            session.execute(sqlalchemy.insert(association_table), batch)
            total_inserted += len(batch)
            batch.clear()

if batch:
    session.execute(sqlalchemy.insert(association_table), batch)
    total_inserted += len(batch)
    batch.clear()

session.commit()

#### 3.8.5 Vinculando Jogos e Linguas

In [40]:
association_table = Game.__table__.metadata.tables["game_language"]

# 1. Carregar all languages e games
languages = {l.id: l for l in session.query(Language).all()}
games = {g.app_id: g for g in session.query(Game).all()}

existing_pairs = set()   # (game_id, language_id)

rows = session.execute(association_table.select())
for game_id, language_id in rows:
    existing_pairs.add((game_id, language_id))

batch = []
total_inserted = 0

for row in games_dataset.itertuples():
    game = games.get(row.appid)
    if not game or not isinstance(row.supported_languages, list):
        continue
    
    for language_id in row.supported_languages:
        if language_id not in languages:
            continue
        
        pair = (game.id, language_id)
        if pair in existing_pairs:
            continue  # já existe, pula
        
        existing_pairs.add(pair)  # marcar como já incluído
        batch.append({"game_id": game.id, "language_id": language_id})
        
        if len(batch) >= BATCH_SIZE:
            session.execute(sqlalchemy.insert(association_table), batch)
            total_inserted += len(batch)
            batch.clear()

if batch:
    session.execute(sqlalchemy.insert(association_table), batch)
    total_inserted += len(batch)
    batch.clear()

session.commit()
